In [1]:
! docker images

REPOSITORY                                                                       TAG       IMAGE ID       CREATED        SIZE
asia-southeast1-docker.pkg.dev/vertexai-intro-347706/mle-docker/containerizeml   latest    af513a508317   4 weeks ago    1.35GB
teyanglau/containerizeml                                                         latest    af513a508317   4 weeks ago    1.35GB
hello-world                                                                      latest    feb5d9fea6a5   8 months ago   13.3kB


In [10]:
%%bash

mkdir -p ../dockerTYtry/src

cat > ../dockerTYtry/src/FaceMaskEfficientNet.py <<CODE

import tensorflow as tf
from tensorflow import keras
import numpy as np
#from PIL import Image
import os
from keras import layers

print(os.getcwd())
print(os.listdir())

#Data is downloaded into the container via shellscript. We believe this is more efficient
traindirectory="/app/data/Train_Small"
testdirectory="/app/data/Test_Small"
image_size=224
TrainData=keras.utils.image_dataset_from_directory(traindirectory, class_names=["WithoutMask","WithMask"], image_size=(image_size,image_size))
TestData=keras.utils.image_dataset_from_directory(testdirectory, class_names=["WithoutMask","WithMask"], image_size=(image_size,image_size))

img_augmentation = keras.models.Sequential(
    [
        layers.RandomRotation(factor=0.15),
        layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
        layers.RandomFlip(),
        layers.RandomContrast(factor=0.1),
    ],
    name="img_augmentation",
)

def build_model(num_classes, IMG_SIZE):
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))

    x = img_augmentation(inputs) #image augmentation within the model. Should this be good practice? Or do we do it inside the map.
    model = keras.applications.EfficientNetB0(include_top=False, input_tensor=x, weights="imagenet")
    
    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)
    

    top_dropout_rate = 0.2
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(num_classes, activation="softmax", dtype='float32', name="pred")(x)

    # Compile
    model = tf.keras.Model(inputs, outputs, name="EfficientNet")
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
    model.compile(
        optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"]
    )
    return model

def normalize_img(image, label):
    return tf.cast(image, tf.float32) / 255., label

from datetime import datetime
strategy = tf.distribute.MirroredStrategy()

TrainData.map(normalize_img).prefetch(tf.data.AUTOTUNE).batch(64*strategy.num_replicas_in_sync)
TestData.map(normalize_img).prefetch(tf.data.AUTOTUNE).batch(64*strategy.num_replicas_in_sync)
logs = "logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")

tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1,
                                                 profile_batch = '500,520')


with strategy.scope():
    model=build_model(2, 224)
model.fit(TrainData,
        epochs=1,
          validation_data=TestData
         , callbacks=[tboard_callback])
model.save("FaceMaskEfficientNetModel")
CODE

In [3]:
# ! python ../dockerTYtry/src/FaceMaskEfficientNet.py

Cant Test Training Code Due to GPU Usage requiring complex setup that is easier to do with Docker /n
Pull Docker Image

Create Requirements Text

In [11]:
%%bash

cat > ../dockerTYtry/requirements.txt <<EOF

EOF

Copy Authentication File into Container. This step can be skipped on GCP as it will be auto-auth (and yes this is not safe)

In [4]:
%%bash
cp daring-hash-348101-2f4dd5ea462e.json ./ml_training_gcp

Create Shell Script to Download Data (Note: The entire folder structure will be copied into app. Therefore /app/FaceMask will exist)

In [5]:
%%bash

cat > ../dockerTYtry/src/initialize.sh <<EOF
#! /bin/sh
gsutil -mq cp -r gs://face-mask-dataset-smu/ /app/data/
python ./src/FaceMaskEfficientNet.py
gsutil -mq cp -r /app/FaceMaskEfficientNetModel gs://face-mask-dataset-smu/Models/
gsutil -mq cp -r /app/logs gs://face-mask-dataset-smu/logs/
rm -r /app/data
EOF

Create Docker File

In [6]:
%%bash

cat > ../dockerTYtry/Dockerfile <<EOF
FROM gcr.io/deeplearning-platform-release/tf-gpu.2-8

WORKDIR /app

COPY . /app
RUN pip install -r requirements.txt

ENTRYPOINT ["sh", "initialize.sh"]
EOF

Build Docker

In [7]:
%%bash

docker build ../dockerTYtry/ -t teyanglau/facemask-ml-train

#2 [internal] load .dockerignore
#2 sha256:fe11ce61aa789f42602c60816d12173f7519e658c342fdbdffa99df3bb9cc8ce
#2 transferring context: 2B 0.0s done
#2 DONE 0.0s

#1 [internal] load build definition from Dockerfile
#1 sha256:3996cb2a1495480f5b9d9777fe182edf1196630cc89955996c82dfd820be3767
#1 transferring dockerfile: 197B 0.0s done
#1 DONE 0.1s

#3 [internal] load metadata for gcr.io/deeplearning-platform-release/tf-gpu.2-8:latest
#3 sha256:38885463e846526fc0e8d585f14202d74ae98ebd5f0945b54469e2c8de63547b
#3 DONE 0.7s

#7 [1/4] FROM gcr.io/deeplearning-platform-release/tf-gpu.2-8@sha256:5bcd6b34a8c00142040d1561b2a39d5ac13ba576bc8c22548d12d98d37ade168
#7 sha256:b7f6c9254a3ddcf119006fe5e72cb779554d7200fb9acec6f721edd0d9d7159b
#7 DONE 0.0s

#8 [internal] load build context
#8 sha256:2da78534fb08d14cc6e4177cd68809abdf93392b597ae788ebb403df0e298383
#8 transferring context: 607B 0.0s done
#8 DONE 0.0s

#4 [2/4] WORKDIR /app
#4 sha256:37a2a63ecebb33c332ba2a3578fe6e1a584a74e3b696a9fbfeefe01a531e2dc

In [15]:
! docker images

REPOSITORY                                                                       TAG       IMAGE ID       CREATED         SIZE
teyanglau/facemask-ml-train                                                      latest    2230858b3e63   4 seconds ago   15.1GB
asia-southeast1-docker.pkg.dev/vertexai-intro-347706/mle-docker/containerizeml   latest    af513a508317   4 weeks ago     1.35GB
teyanglau/containerizeml                                                         latest    af513a508317   4 weeks ago     1.35GB
hello-world                                                                      latest    feb5d9fea6a5   8 months ago    13.3kB


In [8]:
#Run Container

In [ ]:
# run on terminal
# install, authenticate and init gcloud
# https://cloud.google.com/sdk/docs/install#deb 
# https://cloud.google.com/sdk/docs/initializing

gcloud auth login --no-launch-browser # gcloud init --console-only
sudo usermod -a -G docker ${USER} # not needed for MacOS